In [1]:
# Kafka functions
from kafka import KafkaProducer
from kafka import KafkaConsumer

import time
import numpy as np
from json import dumps, loads

In [2]:
#Dash and Plotly functions
import dash
from dash.dependencies import Output, Input
import dash_core_components as dcc
import dash_html_components as html
import plotly
import random
import plotly.graph_objs as go
from collections import deque

#Initialize X and Y axis for live plot
X = deque()
X.append(0)
Y = deque()
Y.append(0)

## Define live graph parameters

In [3]:
app = dash.Dash(__name__)
  
app.layout = html.Div(
    [
        dcc.Graph(id = 'live-graph', animate = True),
        dcc.Interval(
            id = 'graph-update',
            interval = 1000,
            n_intervals = 0
        ),
    ]
)
  
@app.callback(
    Output('live-graph', 'figure'),
    [ Input('graph-update', 'n_intervals') ]
)
  
def update_graph_scatter(n):
    global sentiments_trayectory
    X.append(X[-1]+1)
  
    data = plotly.graph_objs.Scatter(
            x=list(X),
            y=list(sentiments_trayectory),
            name='Scatter',
            mode= 'lines+markers'
    )
  
    return {'data': [data],
            'layout' : go.Layout(xaxis=dict(range=[min(X),max(X)]),yaxis = dict(range = [-100,100]),)}


## Subscribe to topic and define listener function

In [4]:
#Declare consumer
consumer = KafkaConsumer('tweets', auto_offset_reset = 'earliest')

In [5]:
def kafka_listener(time_span,print_tweets,plot_sentiment):
    if plot_sentiment == True:
        global sentiments_trayectory 
    starting = time.time()
    elapsed = 0

    while elapsed < time_span:
        elapsed = time.time() - starting
        for msg in consumer:
            elapsed = time.time() - starting
            dato = msg

            tweet_dict = loads(dato.value)
            user = tweet_dict['user']
            followers = tweet_dict['followers']
            text = tweet_dict['text']
            sentiment = tweet_dict['sentiment']
            subjetivity = tweet_dict['subjetivity']
            if print_tweets == True:
                print('User:',user,', followers:',followers,', sentiment:',sentiment, ', subjetivity:',subjetivity)
            
            if plot_sentiment == True:
                #In order to take account of the following from the user, a weighted sentiment is used for the graph 
                weighted_sentiment = np.log(followers+1) * sentiment
                sentiments_trayectory.append(sentiments_trayectory[-1]+weighted_sentiment)
            if elapsed > time_span:
                return
    return

## Listen tweets

In [ ]:
kafka_listener(20,print_tweets = True,plot_sentiment = False)

## Listen tweets and plot sentiment of tweets 

In [ ]:
from multiprocessing import Process
from multiprocessing import Manager
#A manager proxy list is required in order to add values to the list the plot will be
#reading, both at the same time
manager = Manager()
sentiments_trayectory = manager.list([0])

p = Process(target=kafka_listener, args=(300,True,True))
#Intialize sentiment listening
p.start()
#Run live Graph
app.run_server()
p.join()